In [1]:
import opendatasets as od
od.download("https://www.kaggle.com/datasets/splcher/animefacedataset")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:Your Kaggle Key:Dataset URL: https://www.kaggle.com/datasets/splcher/animefacedataset


100%|██████████| 395M/395M [00:01<00:00, 406MB/s] 


In [2]:
data_dir = './animefacedataset/images'

In [4]:
import torch
import torch.nn as nn

In [5]:
class Discriminator():
    def __init__(self, input_channel, output_channel):
        super().__init__()
        self.layer = nn.Sequential(
            nn.Conv2d(3, 64, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(128, 256, 4, 2, 1, bias=False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(256, 512, 4, 2, 1, bias=False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(512, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
            )
        
    def forward(self, x):
        batch_size, _, _, _ = x.shape
        x = self.layer(x)
        x = x.reshape(batch_size,-1)

In [ ]:
class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer = nn.Sequential(
        nn.ConvTranspose2d(100, 512, 4, 1, 0, bias=False),
        nn.BatchNorm2d(512),
        nn.ReLU(inplace=True),
        nn.ConvTranspose2d(512, 256, 4, 2, 1, bias=False),
        nn.BatchNorm2d(256),
        nn.ReLU(inplace=True),
        nn.ConvTranspose2d(256, 128, 4, 2, 1, bias=False),
        nn.BatchNorm2d(128),
        nn.ReLU(inplace=True),
        nn.ConvTranspose2d(128, 64, 4, 2, 1, bias=False),
        nn.BatchNorm2d(64),
        nn.ReLU(inplace=True),
        nn.ConvTranspose2d(64, 3, 4, 2, 1, bias=False),
        nn.Tanh())

    def forward(self, x):
        return self.layer(x)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
from torch.utils.data import Dataset, DataLoader

In [8]:
import torchvision.transforms as T

In [ ]:
transform = T.Compose([T.Resize((64, 64)),
            T.ToTensor(),
            T.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])]
 )

In [ ]:
class MyDataset(Dataset):
    def __init__(self, real_image, transform):
        self.real_image = real_image
        self.transform = transform

    def __len__(self):
        return self.real_image
    
    def __getitem__(self, idx):
        return self.transform(self.real_image[idx])

In [ ]:
class Model():
    def __init__(self, batch_size = 128, num_epochs = 10, alpha = 0.001, beta = 0.001):
        self.G_model = Generator()
        self.D_model = Discriminator()
        self.G_opt = torch.optim.Adam(self.G_model.parameters(), lr = alpha)
        self.D_opt = torch.optim.Adam(self.D_model.parameters(), lr = beta)
        self.criterion = nn.BCELoss()
        self.batch_size = batch_size
        self.num_epochs = num_epochs

    def train(self):
        for i in range(self.num_epochs):
            for idx, real_image in enumerate(data_loader):
                real_image = real_image.to(device)
                noise = torch.randn(self.batch_size, 100, 1, 1)
                noise = noise.to(device)

                fake_image = self.G_model(noise)

                for params in self.D_model.parameters():
                    params.requires_grad = True

                D_real_pred = self.D_model(real_image)
                D_fake_pred = self.D_model(fake_image.detach())

                D_real_loss = self.criterion(D_real_pred, torch.ones_like(D_real_pred))
                D_fake_loss = self.criterion(D_fake_pred, torch.zeros_like(D_fake_pred))

                D_loss = 0.5 * (D_real_loss + D_fake_loss)

                self.D_opt.zero_grad()

                D_loss.backward()

                self.D_opt.step()

                for params in self.D_model.parameters():
                    params.requires_grad = False

                D_pred_for_gan = self.D_model(fake_image)
                G_loss = self.criterion(D_pred_for_gan, torch.ones_like(D_pred_for_gan))

                self.G_opt.zero_grad()

                G_loss.backward()

                self.G_opt.step()
